In [2]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

# Putting together a PMD database

Making a JSON database of Pokemon Mystery Dungeon RT and Explorers of Sky data from the Bulbapedia pages.

## Data cleaning

In [2]:
pokedex = pd.read_csv('pokedex.csv')

Downloaded a pokedex dataset from kaggle. It has too much useless info, so I'll shrunk it quite a bit.

I'll only need the dex number and the names

In [3]:
pokedex = pokedex[['pokedex_number', 'name']]

No megas or special pokemon!

In [4]:
pokedex = pokedex.drop_duplicates(subset=['pokedex_number'])

In [5]:
pokedex = pokedex.set_index('pokedex_number')

Down to the first 4 generations

In [6]:
pokedex = pokedex[:493]

**final (for now):**

In [7]:
pokedex

,name
pokedex_number,
1,Bulbasaur
2,Ivysaur
3,Venusaur
4,Charmander
5,Charmeleon
...,...
489,Phione
490,Manaphy
491,Darkrai


***

## Scraping data from bulbapedia

List of what is needed:

**gen 4:**
* Explorers of Sky locations
* Recruitment rate

**gen 3:**
* Red/Blue rescue team locations
* Body size
* Recruitment rate
* Friend Area

**both:**
* portrait

In [7]:
import re

In [38]:
# testing the regular expression search, with best poke mudkip
html = requests.get('https://bulbapedia.bulbagarden.net/wiki/Mudkip_(Pok%C3%A9mon)')
soup = BeautifulSoup(html.content, 'html.parser')

sky = soup.find_all(href=re.compile("Explorers_of_Sky"))

In [33]:
rt = soup.find_all(href=re.compile("Red_Rescue_Team"))

**rescue team:** \
`rt[0]` and `rt[1]`: locations\
`rt[2]`: other info

**explorers of sky:** \
`sky[0]`: locations\
`sky[1]`: other info

### LOCATION

#### Rescue Team

In [69]:
rt_loc = rt[1].find_parent().find_next_sibling("td").find_all("a")

In [71]:
rt_loc = [each.text for each in rt_loc]

In [72]:
rt_loc

['Starter Pokémon', 'Partner Pokémon', 'Waterfall Pond']

#### Explorers of Sky

In [73]:
sky_loc = sky[0].find_parent().find_next_sibling("td").find_all("a")

In [75]:
sky_loc = [each.text for each in sky_loc]

In [76]:
sky_loc

['Starter Pokémon', 'Partner Pokémon', 'Lake Afar']

### IMAGE

In [80]:
img = sky[1].find_parent().find_next_sibling().find_next("a")

In [82]:
img['href']

'/wiki/File:MDP_RT_258.png'

### SKY RECRUIT RATE

In [81]:
img.find_next("a").next_sibling.strip().strip(': ')

'8.2%'

### RT RECRUIT RATE

In [78]:
rt[2].find_next("a").next_sibling.strip().strip(': ')

'12.3%'

### BODY SIZE

In [77]:
int(rt[2].find_parent().find_next_sibling().find_next("td").find_next("td").text.strip().strip('Body size: '))

1

### FRIEND AREA

In [88]:
rt[2].find_all_next("a")[2].find_next("span").text

'Peanut Swamp'

***

### Saving the HTML pages locally to save time debugging

In [9]:
for i in range(493):
    pokeName = pokedex['name'].loc[pokedex.index[i]]
    pokeName = pokeName.replace(' ', '_')
    url = 'https://bulbapedia.bulbagarden.net/wiki/{0}_(Pok%C3%A9mon)'.format(pokeName)
    
    html = requests.get(url)
    
    with open('pages/{0}.html'.format(i), "w") as file:
        file.write(html.text)

### Fixing multi forme pokemon

Should've done it before downloading the html files, but oh well

In [19]:
pokedex.loc[pokedex['name']=='Deoxys Normal Forme', 'name'] = 'Deoxys'

In [22]:
pokedex.loc[pokedex['name']=='Shaymin Land Forme', 'name'] = 'Shaymin'

In [24]:
pokedex.loc[pokedex['name']=='Giratina Altered Forme', 'name'] = 'Giratina'

In [34]:
pokedex.loc[pokedex['name']=='Wormadam Plant Cloak', 'name'] = 'Wormadam'

### The actual scraping algorithm

In [90]:
rt_location = list()
sky_location = list()
rt_rate = list()
sky_rate = list()
size = list()
friend_area = list()
image = list()

for i in range(492):
    try:
        with open("pages/{0}.html".format(i), "r") as file:
            soup = BeautifulSoup(file, 'html.parser')
            
        sky = soup.find_all(href=re.compile("Explorers_of_Sky"))
        print("\n------------ {0}\n".format(i))
        
        # some pokemon, like arbok, have more than 2 links. a workaround
        if i == 129 or i == 347 or i == 476:
            sky_index = 2
        elif i == 131 or i == 436 or i == 480 or i == 487 or i == 490:
            sky_index = 1
        elif i==263 or i==291 or i==350 or i==351 or i==411 or i==420 or i==421 or i==422 or i==446 or i==447 or i==478 or i==488:
            sky_index = 0
        else:
            if len(sky) > 2:
                sky_index = len(sky) - 2
            else:
                sky_index = 0
    
        if i < 386:
            rt = soup.find_all(href=re.compile("Red_Rescue_Team"))
            
            # handling some exceptions
            if i == 49 or i == 55 or i == 177 or i == 270 or i == 358:
                rt_index = 3
            elif i == 234 or i == 379:
                rt_index = 2
            elif i == 273:
                rt_index = 0
            else:    
                rt_index = 1

            # rescue team locations
            if i == 196: # umbreon
                rt_loc = ['Evolve', 'Eevee using a', 'Lunar Ribbon']
            elif i == 273: # nuzleaf
                rt_loc = ['Uproar Forest (5F-9F)']
            elif i == 291:
                rt_loc = ['Evolve', 'Nincada']
            elif i == 358:
                rt_loc = ['Frosty Forest,' 'Northwind Field (28F-29F),' 'Western Cave (40F-49F)']
            elif i == 379:
                rt_loc = ['Pitfall Valley']
            else:
                rt_loc = rt[rt_index].find_parent().find_next_sibling("td").find_all("a")
                rt_loc = [(each.text + each.next_sibling.strip()) for each in rt_loc]
            
            rt_location.append(rt_loc)
                
            print("RT Location:", rt_loc)

            # recruit rate
            rt_rate.append(rt[rt_index+1].find_next("a").next_sibling.strip().strip(': '))
            print("R. Rate (RT):", rt_rate[-1])

            # size
            # size.append(int(rt[2].find_parent().find_next_sibling().find_next("td").find_next("td").text.strip().strip('Body size: ')))

            # friend area
            friend_area.append(rt[rt_index+1].find_all_next("a")[2].find_next("span").text)
            print("Friend Area:", friend_area[-1])
        
        # explorers of sky locations
        # Smeargle's page has an error. it can't find the sky locations
        if i == 234:
            sky_location.append(['Foggy Forest (1F-5F)', 'Marine Resort (1F-19F)'])
            sky_index = -1
        elif i == 465: # same with electivire
            sky_location.append(['Spacial Rift (B1F-B15F, Deep Spacial Rift B1F-B9F)'])
            sky_index = -1
        elif i == 483:
            sky_location.append(['Spacial Rift (Bottom)'])
            sky_index = -1
        elif i == 484:
            sky_location.append(['Giant Volcano (Summit)'])
            sky_index = -1
        elif i == 485:
            sky_location.append(['Aegis Cave (Regigigas Chamber) '])
            sky_index = -1
        elif i == 487:
            sky_location.append(['Sharpedo Bluff'])
        else:
            sky_loc = sky[sky_index].find_parent().find_next_sibling("td").find_all("a")
            sky_loc = [(each.text + each.next_sibling.strip()) for each in sky_loc]
            sky_location.append(sky_loc)
        
        print("Sky Location:", sky_location[-1])
        
        # portrait
        if i == 476:
            img = '//archives.bulbagarden.net/media/upload/6/68/MDP_E_477.png'
            image.append(img)
            sky_rate.append('8.2%')
        else:
            img = sky[sky_index+1].find_parent().find_next_sibling().find_next("img")
            image.append(img['src'])
            print("Image ok")

            # sky recruitment rate
            sky_rate.append(img.find_next("a").next_sibling.strip().strip(': '))
            print("R. Rate (sky):", sky_rate[-1])
        
    except Exception as ex:
        if i == 249 or i == 385 or i == 412 or i == 486 or i == 491:
            rt_location.append('')
            rt_rate.append('')
            sky_location.append('')
            sky_rate.append('')
            # size.append('')
            friend_area.append('')
            image.append('')
            continue
            
        print(str(i)+': ', ex)
        break



------------ 0

RT Location: ['Starter Pokémon,', 'Partner Pokémon,', 'Joyous Tower(1F-5F)']
R. Rate (RT): 12.5%
Friend Area: Beau Plains
Sky Location: ['Starter Pokémon,', 'Partner Pokémon,', 'Mystifying Forest(B1-B13),', 'Mystery Jungle(B1-B29)']
Image ok
R. Rate (sky): 8.2%

------------ 1

RT Location: ['Western Cave(B19F-B27F)']
R. Rate (RT): -19%
Friend Area: Beau Plains
Sky Location: ['Mystifying Forest(B1-B13),', 'Sky Peak(1st Station Pass 1F-6th Station Pass 3F),', 'Mystery Jungle(B1-B29)']
Image ok
R. Rate (sky): -10%

------------ 2

RT Location: ['Evolve', 'Ivysaur']
R. Rate (RT): Evolve
Friend Area: Beau Plains
Sky Location: ['Mystery Jungle(1F-29F)']
Image ok
R. Rate (sky): -12%

------------ 3

RT Location: ['Starter Pokémon,', 'Partner Pokémon,', 'Fiery Field(5F-8F)']
R. Rate (RT): 11.7%
Friend Area: Mt. Cleft
Sky Location: ['Starter Pokémon,', 'Partner Pokémon,', 'Dark Crater(B1F-B10F),', 'Giant Volcano(1F-10F)']
Image ok
R. Rate (sky): 8.2%

------------ 4

RT Locati


------------ 39

RT Location: ['Evolve', 'Jigglypuff']
R. Rate (RT): 
Friend Area: MD Time
Sky Location: ['Marine Resort(10F-19F)']
Image ok
R. Rate (sky): -12%

------------ 40

RT Location: ['Lapis Cave(B1F-B3F),', 'Murky Cave(B1F-B5F)']
R. Rate (RT): 13.7%
Friend Area: Echo Cave
Sky Location: ['Aegis Cave(Ice B1F-B3F, Rock B1F-B3F, Steel B1F-B3F, Pit B1F-B3F, B5F)']
Image ok
R. Rate (sky): 5.5%

------------ 41

RT Location: ['Lapis Cave(B11F-B14F),', 'Buried Relic(B1F-B4F),', 'Wish Cave(B46F-B50F),', 'Murky Cave(B6F-B10F)']
R. Rate (RT): (B11F-B14F),
Friend Area: Ranger 
Sky Location: ['Aegis Cave(Ice B1F-B3F, Rock B1F-B3F, Steel B1F-B3F, Pit B1F-B3F, B5F),', 'Labyrinth Cave(B1F-B9F)']
Image ok
R. Rate (sky): -4.5%

------------ 42

RT Location: ['Sinister Woods(1F-4F),', 'Wyvern Hill(13F-20F)']
R. Rate (RT): 12%
Friend Area: Jungle
Sky Location: ['Apple Woods(1F-6F),', 'Oran Forest(1F-4F),', 'Happy Outlook(B1F-B15F)']
Image ok
R. Rate (sky): 8.2%

------------ 43

RT Location: ['


------------ 76

RT Location: ['Joyous Tower(57F-59F)']
R. Rate (RT): 10.9%
Friend Area: Scorched Plains
Sky Location: ['Dark Crater(B1F-B13F),', 'Giant Volcano(1F-8F)']
Image ok
R. Rate (sky): 8.2%

------------ 77

RT Location: ['Evolve', 'Ponyta']
R. Rate (RT): Evolve
Friend Area: Scorched Plains
Sky Location: ['Dark Crater(B4F-B15F, Deep Dark Crater B1F-B9F),', 'Giant Volcano(1F-19F)']
Image ok
R. Rate (sky): -4.5%

------------ 78

RT Location: ['Stormy Sea(B11F-B19F),', 'Silver Trench(B6F-B14F),', 'Grand Sea(B6F-B14F),', 'Far-off Sea(B6F-B14F),', 'Marvelous Sea(B6F-B14F),', 'Fantasy Strait(B6F-B14F)']
R. Rate (RT): 12%
Friend Area: Bountiful Sea
Sky Location: ['Lake Afar(B1F-B20F)']
Image ok
R. Rate (sky): 8.2%

------------ 79

RT Location: ['Evolve', 'Slowpoke']
R. Rate (RT): Evolve
Friend Area: Bountiful Sea
Sky Location: ['Surrounded Sea(B1F-B19F),', 'Miracle Sea(B1F-B18F, Deep Miracle Sea B1F-B3F),', 'Marine Resort(1F-19F)']
Image ok
R. Rate (sky): -12%

------------ 80

RT


------------ 113

RT Location: ['Lapis Cave(9F-12F),', 'Joyous Tower(47F-50F)']
R. Rate (RT): 7.4%
Friend Area: Jungle
Sky Location: ['Waterfall Cave(B1F-B8F)']
Image ok
R. Rate (sky): 6.4%

------------ 114

RT Location: ['Western Cave(13F-19F)']
R. Rate (RT): (13F-19F)
Friend Area: MD Time
Sky Location: ['Mt. Travail(1F-19F)']
Image ok
R. Rate (sky): 6.4%

------------ 115

RT Location: ['Silver Trench(10F-20F),', 'Grand Sea(10F-20F),', 'Fantasy Strait(B1-B10)']
R. Rate (RT): 12.4%
Friend Area: Treasure Sea
Sky Location: ['Surrounded Sea(B1F-B19F),', 'Miracle Sea(B1F-B18F, Deep Miracle Sea B1F-B3F),', 'Bottomless Sea(B1F-B49F)']
Image ok
R. Rate (sky): 8.2%

------------ 116

RT Location: ['Stormy Sea(30F-39F),', 'Silver Trench(30F-39F),', 'Grand Sea(27F-30F),', 'Fantasy Strait(B30)']
R. Rate (RT): -19%
Friend Area: Treasure Sea
Sky Location: ['Surrounded Sea(B1F-B19F),', 'Miracle Sea(B1F-B18F, Deep Miracle Sea B1F-B3F),', 'Bottomless Sea(B1F-B49F)']
Image ok
R. Rate (sky): -10%

--


------------ 152

RT Location: ['Western Cave(20F-27F)']
R. Rate (RT): -19%
Friend Area: Beau Plains
Sky Location: ['Mystifying Forest(B1-B13),', 'Sky Peak(1st Station Pass 1F-6th Station Pass 3F),', 'Mystery Jungle(1F-29F)']
Image ok
R. Rate (sky): 10%

------------ 153

RT Location: ['Bayleef']
R. Rate (RT): 
Friend Area: MD Time
Sky Location: ['Mystery Jungle(1F-29F)']
Image ok
R. Rate (sky): 12%

------------ 154

RT Location: ['Starter Pokémon,', 'Partner Pokémon,', 'Joyous Tower(13F-17F)']
R. Rate (RT): 11.2%
Friend Area: Scorched Plains
Sky Location: ['Starter Pokémon,', 'Partner Pokémon,', 'Dark Crater(B1F-B10F),', 'Giant Volcano(1F-10F)']
Image ok
R. Rate (sky): 8.2%

------------ 155

RT Location: ['Fiery Field(15F-24F)']
R. Rate (RT): -19%
Friend Area: Scorched Plains
Sky Location: ['Dark Crater(B11F-B15F, Deep Dark Crater B1F-B5F),', 'Giant Volcano(1F-19F)']
Image ok
R. Rate (sky): -10%

------------ 156

RT Location: ['Quilava']
R. Rate (RT): Evolve
Friend Area: Scorched 

...


------------ 268

RT Location: ['Evolve', 'Cascoon']
R. Rate (RT): Evolve
Friend Area: Mist-Rise Forest
Sky Location: ['Landslide Cave(B1F-B12F)']
Image ok
R. Rate (sky): 0.5%

------------ 269

RT Location: ['Waterfall Pond(1F-3F)']
R. Rate (RT): 8%
Friend Area: Tadpole Pond
Sky Location: ['Waterfall Cave(B1F-B8F),', 'Serenity River(B1F-B7F)']
Image ok
R. Rate (sky): 6.4%

------------ 270

RT Location: ['Waterfall Pond(13F-16F)']
R. Rate (RT): -19%
Friend Area: Tadpole Pond
Sky Location: ['Lake Afar(B1F-B12F)']
Image ok
R. Rate (sky): -4.5%

------------ 271

RT Location: ['Evolve', 'Lombre']
R. Rate (RT): Evolve
Friend Area: Tadpole Pond
Sky Location: ['Lake Afar(B1F-B12F)']
Image ok
R. Rate (sky): -12%

------------ 272

RT Location: ['Uproar Forest(1F-9F),', 'Southern Cavern(1F-9F)']
R. Rate (RT): 11.2%
Friend Area: Mushroom Forest
Sky Location: ['Mystifying Forest(1F-13F),', 'Mystery Jungle(1F-29F),', 'Happy Outlook(B1F-B15F)']
Image ok
R. Rate (sky): 8.2%

------------ 273

RT 


------------ 307

RT Location: ['Evolve', 'Meditite']
R. Rate (RT): 
Friend Area: Ranger 
Sky Location: ['Mt. Travail(1F-19F),', 'Lost Wilderness(B1F-B17F)']
Image ok
R. Rate (sky): -12%

------------ 308

RT Location: ['Mt. Thunder(1F-4F),', 'Lightning Field(1F-6F),', 'Wish Cave(26F-33F),', 'Oddity Cave(B1-B4)']
R. Rate (RT): 7.8%
Friend Area: Thunder Meadow
Sky Location: [' Far Amp Plains(3F-9F)']
Image ok
R. Rate (sky): 6.4%

------------ 309

RT Location: ['Evolve', 'Electrike']
R. Rate (RT): Evolve
Friend Area: Thunder Meadow
Sky Location: ['Evolve', 'Electrike']
Image ok
R. Rate (sky): -10%

------------ 310

RT Location: ['Wonder Mailcode']
R. Rate (RT): 35%
Friend Area: Thunder Meadow
Sky Location: ['Amp Plains(1F-5F)']
Image ok
R. Rate (sky): 8.2%

------------ 311

RT Location: ['Thunderwave Cave(4F-5F),', 'Lightning Field(7F-19F),', 'Joyous Tower(5F-8F)']
R. Rate (RT): 35%
Friend Area: Thunder Meadow
Sky Location: ['Amp Plains(1F-5F)']
Image ok
R. Rate (sky): 8.2%

--------


------------ 345

RT Location: ['Silver Trench(90F-98F)']
R. Rate (RT): (90F-98F)
Friend Area: MD Darkness
Sky Location: ['Miracle Sea(B1F-B18F, Deep Miracle Sea B1F-B4F)']
Image ok
R. Rate (sky): -4.5%

------------ 346

RT Location: ['Stormy Sea(9F-17F),', 'Silver Trench(14F-23F),', 'Grand Sea(14F-23F),', 'Far-off Sea(14F-23F),', 'Marvelous Sea(B14-B20),', 'Fantasy Strait(B14-B23)']
R. Rate (RT): 8.7%
Friend Area: Treasure Sea
Sky Location: ['Drenched Bluff(B1F-B6F)']
Image ok
R. Rate (sky): 8.2%

------------ 347

RT Location: ['Evolve', 'Anorith']
R. Rate (RT): Evolve
Friend Area: Treasure Sea
Sky Location: ['Mt. Avalanche(1F-19F)']
Image ok
R. Rate (sky): -4.5%

------------ 348

RT Location: ['Wonder Mailcode']
R. Rate (RT): 9.2%
Friend Area: Waterfall Lake
Sky Location: ['Lake Afar(B1F-B12F)']
Image ok
R. Rate (sky): 8.2%

------------ 349

RT Location: ['Evolve', 'Feebas']
R. Rate (RT): Evolve
Friend Area: Waterfall Lake
Sky Location: ['Lake Afar(B21F-B24F)']
Image ok
R. Rate 


------------ 386

Sky Location: ['Starter Pokémon,', 'Partner Pokémon,', 'Mystifying Forest(1F-13F),', 'Mystery Jungle(1F-29F),', 'Happy Outlook(B1F-B15F)']
Image ok
R. Rate (sky): 8.2%

------------ 387

Sky Location: ['Mystifying Forest(1F-13F),', 'Mystery Jungle(1F-29F)']
Image ok
R. Rate (sky): -10%

------------ 388

Sky Location: ['Mystifying Forest(1F-13F),', 'Sky Peak(1st Station Pass 1F-6th Station Pass 3F),', 'Mystery Jungle(1F-29F)']
Image ok
R. Rate (sky): -12%

------------ 389

Sky Location: ['Starter Pokémon,', 'Partner Pokémon,', 'Dark Crater(B1F-B10F),', 'Giant Volcano(1F-7F)']
Image ok
R. Rate (sky): 8.2%

------------ 390

Sky Location: ['Dark Crater(B1F-B15F, Deep Dark Crater B1F-B5F),', 'Giant Volcano(1F-19F)']
Image ok
R. Rate (sky): -10%

------------ 391

Sky Location: ['Deep Dark Crater(B6F-B14F)']
Image ok
R. Rate (sky): -12%

------------ 392

Sky Location: ['Starter Pokémon,', 'Blizzard Island(1F-20F),', 'Crevice Cave(B1F-B10F, Lower Crevice Cave B1F-B4F),'


------------ 462

Sky Location: ['Marine Resort(7F-19F)']
Image ok
R. Rate (sky): -12%

------------ 463

Sky Location: ['Deep Dark Crater(B1F-B14F),', 'Shimmer Desert(5F-8F)']
Image ok
R. Rate (sky): -12%

------------ 464

Sky Location: ['Sky Stairway(1F-49F)']
Image ok
R. Rate (sky): 8.2%

------------ 465

Sky Location: ['Spacial Rift (B1F-B15F, Deep Spacial Rift B1F-B9F)']
Image ok
R. Rate (sky): -12%

------------ 466

Sky Location: ['Deep Dark Crater(B5F-B14F)']
Image ok
R. Rate (sky): -10%

------------ 467

Sky Location: ['Sky Stairway(1F-49F)']
Image ok
R. Rate (sky): -12%

------------ 468

Sky Location: ['Amp Plains(9F-10F, Far Amp Plains 1F-4F)']
Image ok
R. Rate (sky): -10%

------------ 469

Sky Location: ['Sky Stairway(1F-49F)']
Image ok
R. Rate (sky): -10%

------------ 470

Sky Location: ['Crevice Cave(B1F-B10F, Lower Crevice Cave B1F-B4F)']
Image ok
R. Rate (sky): -10%

------------ 471

Sky Location: ['Sky Stairway(1F-49F),', 'Mt. Mistral(1F-19F)']
Image ok
R. Rate

In [95]:
import numpy as np

In [96]:
rt_location += [np.nan] * (len(pokedex) - len(rt_location))
pokedex['rt_location'] = rt_location

In [99]:
rt_rate += [np.nan] * (len(pokedex) - len(rt_rate))
friend_area += [np.nan] * (len(pokedex) - len(friend_area))
pokedex['rt_rate'] = rt_rate
pokedex['friend_area'] = friend_area

In [102]:
pokedex = pokedex[:492]

In [104]:
pokedex['sky_location'] = sky_location
pokedex['sky_rate'] = sky_rate
pokedex['picture'] = image

/tmp/ipykernel_4556/1677968614.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pokedex['sky_location'] = sky_location
/tmp/ipykernel_4556/1677968614.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pokedex['sky_rate'] = sky_rate


In [105]:
len(sky_location)

492

In [106]:
pokedex

,name,rt_location,rt_rate,friend_area,sky_location,sky_rate,picture
pokedex_number,,,,,,,
1,Bulbasaur,"[Starter Pokémon,, Partner Pokémon,, Joyous To...",12.5%,Beau Plains,"[Starter Pokémon,, Partner Pokémon,, Mystifyin...",8.2%,//archives.bulbagarden.net/media/upload/f/fa/M...
2,Ivysaur,[Western Cave(B19F-B27F)],-19%,Beau Plains,"[Mystifying Forest(B1-B13),, Sky Peak(1st Stat...",-10%,//archives.bulbagarden.net/media/upload/3/34/M...
3,Venusaur,"[Evolve, Ivysaur]",Evolve,Beau Plains,[Mystery Jungle(1F-29F)],-12%,//archives.bulbagarden.net/media/upload/8/8c/M...
4,Charmander,"[Starter Pokémon,, Partner Pokémon,, Fiery Fie...",11.7%,Mt. Cleft,"[Starter Pokémon,, Partner Pokémon,, Dark Crat...",8.2%,//archives.bulbagarden.net/media/upload/1/17/M...
5,Charmeleon,[Fiery Field(26F-29F)],-19%,Mt. Cleft,"[Giant Volcano(1F-19F),, Dark Crater(B1F-B15F,...",-10%,//archives.bulbagarden.net/media/upload/d/d2/M...
...,...,...,...,...,...,...,...
488,Cresselia,NaN,NaN,NaN,[Sharpedo Bluff],100%,//archives.bulbagarden.net/media/upload/c/c5/M...
489,Phione,NaN,NaN,NaN,[Miracle Sea(Bottom)],100%,//archives.bulbagarden.net/media/upload/c/cc/M...
490,Manaphy,NaN,NaN,NaN,"[Sharpedo Bluff(after finding, Wonder Eggin, S...",100%,//archives.bulbagarden.net/media/upload/5/5d/M...


### A checkpoint just in case

In [122]:
pokedex.to_csv('PMD_data.csv')

## Post processing fixes

+ Tyranitar, Rayquaza rates are missing
+ Makuhita is missing RT Rate and Friend Area
+ Ho-Oh, Deoxys, Wormadam are lacking everything

In [3]:
pokedex = pd.read_csv('PMD.csv') # needed to load it again bc I closed the notebook

In [11]:
pokedex.loc[247, 'rt_rate'] = 'Evolve'
pokedex.loc[247, 'friend_area'] = 'Mt. Cleft'

In [12]:
pokedex.loc[247]

pokedex_number                                                  248
name                                                      Tyranitar
rt_location                                   ['Evolve', 'Pupitar']
rt_rate                                                      Evolve
friend_area                                               Mt. Cleft
sky_location                            ['Quicksand Pit(B2F-B10F)']
sky_rate                                                       -12%
picture           //archives.bulbagarden.net/media/upload/2/28/M...
Name: 247, dtype: object

In [54]:
pokedex.loc[383, 'rt_rate'] = '-10%'
pokedex.loc[383, 'friend_area'] = 'Stratos Lookout'
pokedex.loc[383, 'sky_rate'] = 'locked 50%'

In [55]:
pokedex.loc[383]

pokedex_number                                                  384
name                                                       Rayquaza
rt_location                                ['Sky TowerSummit (9F)']
rt_rate                                                        -10%
friend_area                                         Stratos Lookout
sky_location                                  ['Sky Stairway(50F)']
sky_rate                                                 locked 50%
picture           //archives.bulbagarden.net/media/upload/8/81/M...
Name: 383, dtype: object

In [46]:
pokedex.loc[295, 'rt_rate'] = '10.2%'
pokedex.loc[295, 'rt_location'] = str(['Buried Relic (30F-34F)'])
pokedex.loc[295, 'sky_location'] = str(['Lost Wilderness (B3F-B16F)'])
pokedex.loc[295, 'friend_area'] = 'Mt. Discipline'
pokedex.loc[295, 'sky_rate'] = '4%'
pokedex.loc[295, 'picture'] = '//archives.bulbagarden.net/media/upload/5/5f/MDP_RT_296.png'

In [47]:
pokedex.loc[295]

pokedex_number                                                  296
name                                                       Makuhita
rt_location                              ['Buried Relic (30F-34F)']
rt_rate                                                       10.2%
friend_area                                          Mt. Discipline
sky_location                         ['Lost Wilderness (B3F-B16F)']
sky_rate                                                         4%
picture           //archives.bulbagarden.net/media/upload/5/5f/M...
Name: 295, dtype: object

In [48]:
pokedex.loc[249, 'rt_location'] = str(['Mt. Faraway (40F)'])
pokedex.loc[249, 'rt_rate'] = '99.9%'
pokedex.loc[249, 'friend_area'] = 'Rainbow Peak'
pokedex.loc[249, 'sky_location'] = str(['Mt. Mistral (19F), (Mystery Part/Secret Slab needed)'])
pokedex.loc[249, 'sky_rate'] = '10%'
pokedex.loc[249, 'picture'] = '//archives.bulbagarden.net/media/upload/3/38/MDP_RT_250.png'

In [49]:
pokedex.loc[249]

pokedex_number                                                  250
name                                                          Ho-oh
rt_location                                   ['Mt. Faraway (40F)']
rt_rate                                                       99.9%
friend_area                                            Rainbow Peak
sky_location      ['Mt. Mistral (19F), (Mystery Part/Secret Slab...
sky_rate                                                        10%
picture           //archives.bulbagarden.net/media/upload/3/38/M...
Name: 249, dtype: object

In [50]:
pokedex.loc[385, 'rt_location'] = str(['Meteor Cave (20F)'])
pokedex.loc[385, 'rt_rate'] = '-10% (all formes)'
pokedex.loc[385, 'friend_area'] = 'Enclosed Island' 
pokedex.loc[385, 'sky_location'] = str(['Shimmer Hill (17F), (Mystery Part/Secret Slab needed)']) 
pokedex.loc[385, 'sky_rate'] = '-10% (all formes)' 
pokedex.loc[385, 'picture'] = '//archives.bulbagarden.net/media/upload/1/19/MDP_RT_386.png'

In [51]:
pokedex.loc[385]

pokedex_number                                                  386
name                                                         Deoxys
rt_location                                   ['Meteor Cave (20F)']
rt_rate                                           -10% (all formes)
friend_area                                         Enclosed Island
sky_location      ['Shimmer Hill (17F), (Mystery Part/Secret Sla...
sky_rate                                          -10% (all formes)
picture           //archives.bulbagarden.net/media/upload/1/19/M...
Name: 385, dtype: object

In [43]:
ls = ['Plant Cloak: Crystal Crossing (B1F-B8F)', 'Sandy Cloak: Crystal Cave (B1F-B11F)', 'Trash Cloak: Landslide Cave (B1F-B12F)']
pokedex.loc[412, 'sky_location'] = str(ls)

In [61]:
pokedex.loc[412, 'sky_rate'] = '8.2%'
pokedex.loc[412, 'picture'] = '//archives.bulbagarden.net/media/upload/d/d6/MDP_E_413.png'

In [62]:
pokedex.loc[412]

pokedex_number                                                  413
name                                                       Wormadam
rt_location                                                     NaN
rt_rate                                                         NaN
friend_area                                                     NaN
sky_location      ['Plant Cloak: Crystal Crossing (B1F-B8F)', 'S...
sky_rate                                                       8.2%
picture           //archives.bulbagarden.net/media/upload/d/d6/M...
Name: 412, dtype: object

In [56]:
pokedex.loc[486, 'sky_location'] = str(['World Abyss (Pit)'])
pokedex.loc[486, 'sky_rate'] = 'locked 50%'
pokedex.loc[486, 'picture'] = '//archives.bulbagarden.net/media/upload/e/e6/MDP_E_487.png'

In [57]:
pokedex.loc[486]

pokedex_number                                                  487
name                                                       Giratina
rt_location                                                     NaN
rt_rate                                                         NaN
friend_area                                                     NaN
sky_location                                  ['World Abyss (Pit)']
sky_rate                                                 locked 50%
picture           //archives.bulbagarden.net/media/upload/e/e6/M...
Name: 486, dtype: object

In [58]:
pokedex.loc[491, 'sky_location'] = str(['Sky Peak (Top)'])
pokedex.loc[491, 'sky_rate'] = '100%'
pokedex.loc[491, 'picture'] = '//archives.bulbagarden.net/media/upload/c/cd/MDP_E_492.png'

In [59]:
pokedex.loc[491]

pokedex_number                                                  492
name                                                        Shaymin
rt_location                                                     NaN
rt_rate                                                         NaN
friend_area                                                     NaN
sky_location                                     ['Sky Peak (Top)']
sky_rate                                                       100%
picture           //archives.bulbagarden.net/media/upload/c/cd/M...
Name: 491, dtype: object

### Other NaN values and broken info

In [65]:
pokedex.loc[8, 'rt_rate'] = '-33.9%'
pokedex.loc[8, 'friend_area'] = 'Turtleshell Pond'

pokedex.loc[9, 'rt_rate'] = '10.2%'
pokedex.loc[9, 'friend_area'] = 'Mist-Rise Forest'

pokedex.loc[10, 'rt_rate'] = '-19%'
pokedex.loc[10, 'friend_area'] = 'Mist-Rise Forest'

pokedex.loc[11, 'rt_rate'] = 'Evolve'
pokedex.loc[11, 'friend_area'] = 'Mist-Rise Forest'

In [67]:
pokedex.loc[5, 'rt_rate'] = 'Evolve'
pokedex.loc[5, 'friend_area'] = 'Mt. Cleft'

pokedex.loc[22, 'rt_rate'] = '12.4%'
pokedex.loc[22, 'friend_area'] = 'Wild Plains'

pokedex.loc[25, 'rt_rate'] = 'Evolve'
pokedex.loc[25, 'friend_area'] = 'Energetic Forest'

pokedex.loc[37, 'rt_rate'] = 'Evolve'
pokedex.loc[37, 'friend_area'] = 'Darkness Ridge'

pokedex.loc[39, 'rt_rate'] = 'Evolve'
pokedex.loc[39, 'friend_area'] = 'Sky Blue Plains'

In [72]:
pokedex[pokedex.isnull().any(axis=1)].head(25)

,pokedex_number,name,rt_location,rt_rate,friend_area,sky_location,sky_rate,picture
50,51,Dugtrio,"['Evolve', 'Diglett']",NaN,Ranger,['Shimmer Desert(1F-9F)'],-4.5%,//archives.bulbagarden.net/media/upload/b/bd/M...
52,53,Persian,"['Evolve', 'Meowth']",NaN,MD Time,['Happy Outlook(B1F-B15F)'],-12%,//archives.bulbagarden.net/media/upload/3/33/M...
64,65,Alakazam,"['Evolve', 'Kadabra']",NaN,MD Time,['Treeshroud Forest(10F-20F)'],-12%,//archives.bulbagarden.net/media/upload/7/73/M...
93,94,Gengar,"['Evolve', 'Haunter']",NaN,Ranger,['Sky Stairway(30F-49F)'],-10%,//archives.bulbagarden.net/media/upload/3/3c/M...
104,105,Marowak,"['Evolve', 'Cubone']",NaN,MD Time,['Sky Peak(8th Station Pass 1F - Sky Peak Summ...,-4.5%,//archives.bulbagarden.net/media/upload/b/bc/M...
111,112,Rhydon,"['Evolve', 'Rhyhorn']",NaN,Ranger,['Sky Stairway(1F-10F)'],-10%,//archives.bulbagarden.net/media/upload/c/c6/M...
143,144,Articuno,['Frosty Grotto(5F)'],(5F),MD Darkness,['Mt. Avalanche(Peak)'],NaN,//archives.bulbagarden.net/media/upload/7/7d/M...
150,151,Mew,['Buried Relic(36F-98F)'],0.9%,Final Island,['Mystery Jungle'],NaN,//archives.bulbagarden.net/media/upload/9/95/M...
153,154,Meganium,['Bayleef'],NaN,MD Time,['Mystery Jungle(1F-29F)'],12%,//archives.bulbagarden.net/media/upload/a/aa/M...
188,189,Jumpluff,"['Evolve', 'Skiploom']",NaN,MD Time,"['Mt. Mistral(1F-19F),', 'Sky Stairway(1F-49F)']",-10%,//archives.bulbagarden.net/media/upload/8/84/M...


In [73]:
pokedex.loc[50, 'rt_rate'] = 'Evolve'
pokedex.loc[50, 'friend_area'] = 'Boulder Cave'

pokedex.loc[52, 'rt_rate'] = 'Evolve'
pokedex.loc[52, 'friend_area'] = 'Energetic Forest'

pokedex.loc[64, 'rt_rate'] = 'Evolve'
pokedex.loc[64, 'friend_area'] = 'Decrepit Lab'

pokedex.loc[93, 'rt_rate'] = 'Evolve'
pokedex.loc[93, 'friend_area'] = 'Darkness Ridge'

pokedex.loc[104, 'rt_rate'] = 'Evolve'
pokedex.loc[104, 'friend_area'] = 'Mt. Deepgreen'

In [77]:
pokedex.loc[111, 'rt_rate'] = 'Evolve'
pokedex.loc[111, 'friend_area'] = 'Safari'

pokedex.loc[114, 'rt_rate'] = '7.6%'
pokedex.loc[114, 'friend_area'] = 'Safari'

pokedex.loc[143, 'rt_rate'] = '30%'
pokedex.loc[143, 'friend_area'] = 'Legendary Island'

pokedex.loc[144, 'rt_rate'] = '30%'
pokedex.loc[144, 'friend_area'] = 'Legendary Island'

pokedex.loc[145, 'rt_rate'] = '30%'
pokedex.loc[145, 'friend_area'] = 'Legendary Island'

In [79]:
pokedex.loc[111, 'sky_rate'] = '30%'

pokedex.loc[149, 'rt_rate'] = '99.9%'
pokedex.loc[149, 'friend_area'] = 'Cryptic Cave'

pokedex.loc[150, 'sky_rate'] = '40%'

pokedex.loc[153, 'rt_rate'] = 'Evolve'
pokedex.loc[153, 'friend_area'] = 'Beau Plains'

pokedex.loc[188, 'rt_rate'] = 'Evolve'
pokedex.loc[188, 'friend_area'] = 'Beau Plains'

In [81]:
pokedex.loc[200, 'rt_rate'] = '1.8% (A-Z), 0.1% (?!)'

pokedex.loc[201, 'rt_rate'] = 'Evolve'
pokedex.loc[201, 'rt_rate'] = 'Echo Cave'

pokedex.loc[223, 'rt_rate'] = '-30%'
pokedex.loc[223, 'friend_area'] = 'Treasure Sea'

pokedex.loc[226, 'rt_rate'] = '6.8%'
pokedex.loc[226, 'friend_area'] = 'Ravaged Field'

In [84]:
pokedex.loc[251, 'rt_rate'] = 'Automatically after completing purity forest'

pokedex.loc[274, 'rt_rate'] = '-30%'
pokedex.loc[274, 'friend_area'] = 'Mushroom Forest'

pokedex.loc[278, 'rt_rate'] = 'Evolve'
pokedex.loc[278, 'friend_area'] = 'Shallow Beach'

pokedex.loc[281, 'rt_rate'] = '1%'
pokedex.loc[281, 'friend_area'] = 'Sky Blue Plains'

pokedex.loc[291, 'rt_rate'] = 'Evolve'
pokedex.loc[291, 'friend_area'] = 'Secretive Forest'
pokedex.loc[291, 'sky_rate'] = '-12%'

In [87]:
pokedex.loc[307, 'rt_rate'] = 'Evolve'
pokedex.loc[307, 'friend_area'] = 'Mt. Discipline'

pokedex.loc[327, 'rt_rate'] = '7.2%'
pokedex.loc[237, 'friend_area'] = 'Mt. Deepgreen'

pokedex.loc[339, 'rt_rate'] = 'Evolve'
pokedex.loc[339, 'friend_area'] = 'Peanut Swamp'

pokedex.loc[345, 'rt_rate'] = '-30%'
pokedex.loc[345, 'friend_area'] = 'Deep-Sea Floor'

In [88]:
pokedex[370:383]

,pokedex_number,name,rt_location,rt_rate,friend_area,sky_location,sky_rate,picture
370,371,Bagon,"['Lapis Cave(9F-13F),', 'Wish Cave(45F-49F),',...",7.6%,Dragon Cave,"['Crystal Crossing(B1F-B7F),', 'Final Maze(B1F...",6.4%,//archives.bulbagarden.net/media/upload/0/0f/M...
371,372,Shelgon,"['Mt. Freeze(1F-6F),', 'Wish Cave(65F-69F),', ...",-19%,Dragon Cave,['Sky Stairway(1F-49F)'],0.5%,//archives.bulbagarden.net/media/upload/c/c4/M...
372,373,Salamence,"['Evolve', 'Shelgon']",Evolve,Dragon Cave,"['Evolve', 'Shelgon']",-10%,//archives.bulbagarden.net/media/upload/f/f8/M...
373,374,Beldum,"['Mt. Steel(6F-8F),', 'Wish Cave(15F-17F),', '...",7.8%,Magnetic Quarry,['Crystal Cave(B1F-B7F)'],6.4%,//archives.bulbagarden.net/media/upload/9/97/M...
374,375,Metang,"['Frosty Grotto(1F-4F),', 'Wish Cave(62F-64F),...",-19%,Magnetic Quarry,['Sky Stairway(1F-49F)'],0.5%,//archives.bulbagarden.net/media/upload/f/f6/M...
375,376,Metagross,"['Evolve', 'Metang']",Evolve,Magnetic Quarry,"['Evolve', 'Metang']",-10%,//archives.bulbagarden.net/media/upload/f/f2/M...
376,377,Regirock,['Buried Relic(15F)'],0.1%,Ancient Relic,['Aegis Cave(Rock Chamber)'],NaN,//archives.bulbagarden.net/media/upload/f/f9/M...
377,378,Regice,['Buried Relic(25F)'],0.1%,Ancient Relic,['Aegis Cave(Ice Chamber)'],NaN,//archives.bulbagarden.net/media/upload/7/70/M...
378,379,Registeel,['Buried Relic(35F)'],0.1%,Ancient Relic,['Aegis Cave(Steel Chamber)'],NaN,//archives.bulbagarden.net/media/upload/2/22/M...
379,380,Latias,['Pitfall Valley'],NaN,Southern Island,"['Spacial Rift(', 'Happy Outlook(']",0.1%,//archives.bulbagarden.net/media/upload/3/33/M...


In [89]:
pokedex.loc[376, 'sky_rate'] = 'locked 50%'
pokedex.loc[377, 'sky_rate'] = 'locked 50%'
pokedex.loc[378, 'sky_rate'] = 'locked 50%'

pokedex.loc[379, 'rt_rate'] = 'Asks to join the team after being rescued'
pokedex.loc[380, 'rt_rate'] = 'Asks to join the team after rescuing Latias'
pokedex.loc[380, 'friend_area'] = 'Southern Island'

pokedex.loc[381, 'sky_rate'] = 'locked 50%'

pokedex.loc[382, 'rt_rate'] = '-10%'
pokedex.loc[382, 'friend_area'] = 'Volcanic Pit'
pokedex.loc[382, 'sky_rate'] = 'locked 50%'

In [91]:
pokedex[pokedex['sky_rate'].isnull()]

,pokedex_number,name,rt_location,rt_rate,friend_area,sky_location,sky_rate,picture
143,144,Articuno,['Frosty Grotto(5F)'],30%,Legendary Island,['Mt. Avalanche(Peak)'],NaN,//archives.bulbagarden.net/media/upload/7/7d/M...
484,485,Heatran,NaN,NaN,NaN,['Giant Volcano (Summit)'],NaN,//archives.bulbagarden.net/media/upload/6/64/M...


In [93]:
pokedex.loc[143, 'sky_rate'] = 'locked 50%'
pokedex.loc[484, 'sky_rate'] = 'locked 50%'
pokedex.loc[484, 'sky_location'] = "['Giant Volcano (Summit)']"

## Final tweaks

In [95]:
pokedex = pokedex.set_index('pokedex_number')

In [96]:
pokedex

,name,rt_location,rt_rate,friend_area,sky_location,sky_rate,picture
pokedex_number,,,,,,,
1,Bulbasaur,"['Starter Pokémon,', 'Partner Pokémon,', 'Joyo...",12.5%,Beau Plains,"['Starter Pokémon,', 'Partner Pokémon,', 'Myst...",8.2%,//archives.bulbagarden.net/media/upload/f/fa/M...
2,Ivysaur,['Western Cave(B19F-B27F)'],-19%,Beau Plains,"['Mystifying Forest(B1-B13),', 'Sky Peak(1st S...",-10%,//archives.bulbagarden.net/media/upload/3/34/M...
3,Venusaur,"['Evolve', 'Ivysaur']",Evolve,Beau Plains,['Mystery Jungle(1F-29F)'],-12%,//archives.bulbagarden.net/media/upload/8/8c/M...
4,Charmander,"['Starter Pokémon,', 'Partner Pokémon,', 'Fier...",11.7%,Mt. Cleft,"['Starter Pokémon,', 'Partner Pokémon,', 'Dark...",8.2%,//archives.bulbagarden.net/media/upload/1/17/M...
5,Charmeleon,['Fiery Field(26F-29F)'],-19%,Mt. Cleft,"['Giant Volcano(1F-19F),', 'Dark Crater(B1F-B1...",-10%,//archives.bulbagarden.net/media/upload/d/d2/M...
...,...,...,...,...,...,...,...
488,Cresselia,NaN,NaN,NaN,['Sharpedo Bluff'],100%,//archives.bulbagarden.net/media/upload/c/c5/M...
489,Phione,NaN,NaN,NaN,['Miracle Sea(Bottom)'],100%,//archives.bulbagarden.net/media/upload/c/cc/M...
490,Manaphy,NaN,NaN,NaN,"['Sharpedo Bluff(after finding', 'Wonder Eggin...",100%,//archives.bulbagarden.net/media/upload/5/5d/M...


In [115]:
pokedex.loc[4,'rt_location'].strip('[]').replace("'", "").replace(",,", ",")

'Starter Pokémon, Partner Pokémon, Fiery Field(5F-8F)'

In [116]:
for i in range(1, 387):
    pokedex.loc[i, 'rt_location'] = pokedex.loc[i, 'rt_location'].strip('[]').replace("'", "").replace(",,", ",")

In [118]:
for i in range(1, 493):
    pokedex.loc[i, 'sky_location'] = pokedex.loc[i, 'sky_location'].strip('[]').replace("'", "").replace(",,", ",")

In [119]:
pokedex

,name,rt_location,rt_rate,friend_area,sky_location,sky_rate,picture
pokedex_number,,,,,,,
1,Bulbasaur,"Starter Pokémon, Partner Pokémon, Joyous Tower...",12.5%,Beau Plains,"Starter Pokémon, Partner Pokémon, Mystifying F...",8.2%,//archives.bulbagarden.net/media/upload/f/fa/M...
2,Ivysaur,Western Cave(B19F-B27F),-19%,Beau Plains,"Mystifying Forest(B1-B13), Sky Peak(1st Statio...",-10%,//archives.bulbagarden.net/media/upload/3/34/M...
3,Venusaur,"Evolve, Ivysaur",Evolve,Beau Plains,Mystery Jungle(1F-29F),-12%,//archives.bulbagarden.net/media/upload/8/8c/M...
4,Charmander,"Starter Pokémon, Partner Pokémon, Fiery Field(...",11.7%,Mt. Cleft,"Starter Pokémon, Partner Pokémon, Dark Crater(...",8.2%,//archives.bulbagarden.net/media/upload/1/17/M...
5,Charmeleon,Fiery Field(26F-29F),-19%,Mt. Cleft,"Giant Volcano(1F-19F), Dark Crater(B1F-B15F, D...",-10%,//archives.bulbagarden.net/media/upload/d/d2/M...
...,...,...,...,...,...,...,...
488,Cresselia,NaN,NaN,NaN,Sharpedo Bluff,100%,//archives.bulbagarden.net/media/upload/c/c5/M...
489,Phione,NaN,NaN,NaN,Miracle Sea(Bottom),100%,//archives.bulbagarden.net/media/upload/c/cc/M...
490,Manaphy,NaN,NaN,NaN,"Sharpedo Bluff(after finding, Wonder Eggin, Su...",100%,//archives.bulbagarden.net/media/upload/5/5d/M...


## Saving the JSON

In [120]:
pokedex.to_json('PMD_data.json', orient='index')

Finished. Did it take less time than it would if I had to type it one by one? Probably. Will never do this again though.